In [1]:
from gensim import corpora,models,similarities
from sklearn.datasets import fetch_20newsgroups as getData
from sklearn.model_selection import train_test_split
from collections import defaultdict
import re
import numpy as np

In [2]:
corpus = getData(subset='train', 
                             remove=('headers', 'footers', 'quotes'),
                            categories=['alt.atheism','comp.graphics',
                                       'soc.religion.christian'])
X = corpus.data
y = corpus.target
y_names = corpus.target_names

In [3]:
print(X[0],len(X),y_names[y[0]])



I would rather be at a higher risk of being killed than actually killed by
                              ^^^^                      ^^^^^^^^
mistake.  Though I do agree with the concept that the type D and E murderers
are a massive waste of space and resources I don't agree with the concept:

	killing is wrong
	if you kill we will punish you
	our punishment will be to kill you.

Seems to be lacking in consistency.
 1663 alt.atheism


In [7]:
type(X[0])

str

In [4]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.1,shuffle=True)

In [5]:
stoplist = set('for a of the and to in'.split(' '))
texts = [[word for word in re.split('\W+',doc.lower()) if word not in stoplist]
         for doc in Xtrain]

In [6]:
print(len(texts))

1496


In [7]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

cutOffValue = 10
processed_corpus = [[token for token in text if frequency[token] > cutOffValue] 
                    for text in texts]

In [8]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(2886 unique tokens: ['', '3', 'able', 'any', 'are']...)


In [9]:
print(dictionary.token2id)

{'': 0, '3': 1, 'able': 2, 'any': 3, 'are': 4, 'at': 5, 'because': 6, 'become': 7, 'been': 8, 'bible': 9, 'claim': 10, 'define': 11, 'defined': 12, 'do': 13, 'during': 14, 'future': 15, 'get': 16, 'goals': 17, 'guess': 18, 'has': 19, 'have': 20, 'i': 21, 'is': 22, 'language': 23, 'languages': 24, 'least': 25, 'lot': 26, 'main': 27, 'many': 28, 'not': 29, 'one': 30, 'or': 31, 'organizations': 32, 'part': 33, 'people': 34, 'provided': 35, 'reason': 36, 'rest': 37, 'rules': 38, 'see': 39, 'simply': 40, 'some': 41, 'still': 42, 'support': 43, 'that': 44, 'their': 45, 'them': 46, 'there': 47, 'they': 48, 'this': 49, 'version': 50, 'we': 51, 'which': 52, 'who': 53, 'will': 54, 'without': 55, 'world': 56, 'write': 57, 'writing': 58, 'written': 59, 'yet': 60, 'accept': 61, 'adam': 62, 'all': 63, 'allow': 64, 'also': 65, 'an': 66, 'anyway': 67, 'based': 68, 'beginning': 69, 'burden': 70, 'but': 71, 'choice': 72, 'course': 73, 'eve': 74, 'fall': 75, 'former': 76, 'free': 77, 'from': 78, 'give': 

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [11]:
print(bow_corpus[100])

[(0, 2), (1, 1), (3, 2), (4, 9), (5, 2), (6, 1), (8, 1), (9, 8), (13, 1), (19, 1), (20, 5), (21, 4), (22, 9), (25, 1), (28, 1), (29, 8), (31, 5), (34, 2), (36, 2), (38, 1), (41, 1), (44, 12), (47, 5), (48, 4), (49, 2), (52, 1), (53, 2), (56, 2), (58, 1), (63, 4), (65, 1), (71, 3), (78, 2), (82, 12), (86, 1), (91, 4), (94, 2), (99, 1), (101, 6), (102, 1), (107, 1), (108, 1), (111, 3), (112, 1), (113, 1), (114, 1), (115, 3), (117, 1), (119, 10), (120, 4), (128, 1), (131, 7), (136, 1), (137, 3), (150, 1), (154, 2), (155, 2), (159, 1), (165, 1), (171, 2), (173, 3), (180, 21), (181, 3), (186, 1), (189, 1), (190, 2), (198, 4), (200, 4), (210, 5), (213, 1), (214, 1), (215, 3), (216, 1), (220, 11), (224, 1), (225, 1), (230, 1), (233, 1), (236, 1), (237, 3), (238, 1), (239, 1), (253, 1), (256, 1), (261, 1), (264, 2), (282, 1), (285, 1), (289, 1), (302, 4), (303, 2), (313, 1), (322, 2), (323, 3), (325, 4), (336, 1), (343, 1), (355, 1), (356, 1), (359, 2), (365, 1), (385, 2), (393, 1), (395, 3), 

In [12]:
model = models.TfidfModel(bow_corpus)
#model = models.LsiModel(bow_corpus)
#model = models.LdaModel(bow_corpus)

In [13]:
index = similarities.SparseMatrixSimilarity(model[bow_corpus],num_features=len(dictionary))

In [21]:
t = np.random.randint(len(Xtest))
query_document = re.split('\W+',Xtest[t].lower())
query_bow = dictionary.doc2bow(query_document)
sims = index[model[query_bow]]
docNumber = sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[0][0]
print('Predicted:',y_names[ytrain[docNumber]])
print('Ground Truth:',y_names[ytest[t]])

Predicted: comp.graphics
Ground Truth: comp.graphics
